Final training, without validation set and no debug logging or plotting.

See Combined.ipynb for more details. 

In [1]:
import numpy as np
import cv2
from google.colab import drive
import gc
from sklearn.model_selection import train_test_split
import tensorflow as tf

import os

from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers.convolutional import Conv2D
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
IMG_DIM = (150,150)
EPOCHS = 2
LEARNING_RATE = 0.0003
BATCH_SIZE = 32
VAL_SPLIT = 0
SHUFFLE = True

SIZE_KITTI = 2555
SIZE_2k19 = 25000

In [3]:
drive.mount('/content/drive/')
root_kitti = "drive/MyDrive/Colab Notebooks/commaai/data/KITTI/"
root_2k19 = "drive/MyDrive/Colab Notebooks/commaai/data/2k19/"

Mounted at /content/drive/


In [4]:
#image prep functions

def opticalFlow(frame1,frame2, x, w, y, h):
    frame1 = frame1[y:y+h, x:x+w]
    frame2 = frame2[y:y+h, x:x+w]

    flow = np.zeros_like(frame1)
    frame1 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    frame2 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow_data = cv2.calcOpticalFlowFarneback(frame1, frame2, None, 0.4, 1, 12, 2, 8, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow_data[...,0], flow_data[...,1])
    mag = (mag *15).astype(int)
    ang = ang*180/np.pi/2
    flow[...,1] = 255
    flow[...,0] = ang
    flow[...,2] = mag
    return flow

def getImagesFrom2k19(path):
    images = []
    cap = cv2.VideoCapture(path)
    ret, prev = cap.read()
    while True:
        ret, next  = cap.read()
        if not ret:
            break
        flow = opticalFlow(prev,next, 9, 1146, 213, 382)
        flow = cv2.resize(flow, IMG_DIM, interpolation = cv2.INTER_AREA)
        images.append(flow/127.5 - 1)
        prev = next
    images = np.array(images)
    return images

def getImagesFromMain(path):
    images = []
    cap = cv2.VideoCapture(path)
    ret, prev = cap.read()
    while True:
        ret, next  = cap.read()
        if not ret:
            break
        flow = opticalFlow(prev,next, 5, 630, 140, 210)
        flow = cv2.resize(flow, IMG_DIM, interpolation = cv2.INTER_AREA)
        images.append(flow/127.5 - 1)
        prev = next
    images = np.array(images)
    return images

def getImagesFromKitti(path, size):
    images = []
    prev = cv2.imread(path+"0000000000.png")
    for i in range(1, size):
        try:
            fn = str(i).zfill(10)+".png"
            next = cv2.imread(path+fn)
            flow = opticalFlow(prev,next, 59, 1125, 0, 375)
            flow = cv2.resize(flow, IMG_DIM, interpolation = cv2.INTER_AREA)
            images.append(flow/127.5 - 1)
            prev = next
        except:
            break
    images = np.array(images)
    return images
    
gc.collect()

15

In [5]:
#label prep functions

def getLabelsFromTXT(path):
    labels = []
    f = open(path,'r')
    for l in f:
        labels.append(float(l))
    labels = np.array(labels)
    return labels

# 2k19 saves timestamaps disconnected to each frame
def getLabelsFrom2k19(Y_path, t_path):
    values = np.load(Y_path).flatten()
    times = np.load(t_path).flatten()
    Y = []
    count = 0
    for i, t in enumerate(times):
        if float(t) - float(times[0]) > count:
            Y.append(values[i])
            count+= (1/20)
    return np.array(Y)

gc.collect()

98

In [ ]:
def getModel():
    model = Sequential()
    model.add(Conv2D(filters=24, kernel_size=(5, 5), strides=(2, 2),activation='relu'))
    model.add(Conv2D(filters=36, kernel_size=(5, 5),strides=(2, 2), activation='relu'))
    model.add(Conv2D(filters=48, kernel_size=(5, 5), strides=(2, 2),activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3) ,activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dense(units=1))

    model.compile(loss='mse',optimizer= tf.keras.optimizers.Adam(lr=LEARNING_RATE))
    return model
    
gc.collect()

22

In [ ]:
X_kitti = np.array([]).reshape(0, IMG_DIM[0], IMG_DIM[1], 2)
Y_kitti = np.array([])

folders_kitti = os.listdir(root_kitti)
count = 0

def getMoreKitti():
    global Y_kitti
    global X_kitti
    global folders_kitti
    global count
    if count < len(folders_kitti):
        folder_id = folders_kitti[count]
        count += 1
        print("In dir:", root_kitti + folder_id)

        X_path = root_kitti + folder_id + "/image_03/data/"
        Y_path = root_kitti + folder_id + "/labels.txt"
        num_photos = len(os.listdir(X_path))

        Y_kitti_batch = getLabelsFromTXT(Y_path)
        
        X_kitti_batch = getImagesFromKitti(X_path, num_photos)
        X_kitti_batch = X_kitti_batch[:,:,:,[0,2]]

        #double the speed because fps is half (10 not 20)
        Y_kitti_batch*=2
        Y_kitti_batch = Y_kitti_batch[:len(X_kitti_batch)]

        if len(Y_kitti_batch) == len(X_kitti_batch):
            return (X_kitti_batch, Y_kitti_batch)
        else:
            X_kitti_batch = X_kitti_batch[:len(Y_kitti_batch)]
            return (X_kitti_batch, Y_kitti_batch)
    return (np.array([]).reshape(0, IMG_DIM[0], IMG_DIM[1], 2))

def getAllKitti():
    while True:
        X_kitti_batch, Y_kitti_batch = getMoreKitti()
        if len(Y_kitti_batch) == 0:
            break
        X_kitti = np.concatenate((X_kitti, X_kitti_batch))
        Y_kitti = np.concatenate((Y_kitti, Y_kitti_batch))
        del X_kitti_batch, Y_kitti_batch
        gc.collect()


def getKittiBatch():
    global Y_kitti
    global X_kitti
    global folders_kitti
    global count
    while len(Y_kitti) < SIZE_KITTI:
        X_kitti_batch, Y_kitti_batch = getMoreKitti()
        if len(Y_kitti_batch) == 0:
            break
        X_kitti = np.concatenate((X_kitti, X_kitti_batch))
        Y_kitti = np.concatenate((Y_kitti, Y_kitti_batch))
        del X_kitti_batch, Y_kitti_batch
        gc.collect()
    X_ret = X_kitti[:SIZE_KITTI]
    Y_ret = Y_kitti[:SIZE_KITTI]
    X_kitti = X_kitti[SIZE_KITTI:]
    Y_kitti = Y_kitti[SIZE_KITTI:]
    gc.collect()
    return (X_ret, Y_ret)

gc.collect()

0

In [ ]:
folder_chunk = "Chunk_"
chunks = 6
target_mean = 13.18

X = np.array([]).reshape(0, IMG_DIM[0], IMG_DIM[1], 2)
Y = np.array([])

model = getModel()


for chunk in range(1,chunks+1):
    for folder_id in os.listdir(root_2k19+folder_chunk+str(chunk)):
        for folder_number in os.listdir(root_2k19+folder_chunk+str(chunk)+"/"+folder_id):
            path = root_2k19+folder_chunk+str(chunk)+"/"+folder_id+"/"+folder_number

            if (7 < int(folder_id[29 : 31]) < 18):
                try:
                    labels_path = path+"/processed_log/CAN/speed/value"
                    times_path = path+"/processed_log/CAN/speed/t"
                    video_path = path+"/video.hevc"

                    Y_batch = getLabelsFrom2k19(labels_path, times_path)
                    if (len(Y) == 0 or (Y.mean() > target_mean and Y_batch.mean() < target_mean) or (Y.mean() < target_mean and Y_batch.mean() > target_mean)):
                        X_batch = getImagesFrom2k19(video_path)

                        X_batch = X_batch[:,:,:,[0,2]]

                        Y_batch = Y_batch[:len(X_batch)]

                        if len(Y_batch) == len(X_batch):
                            X = np.concatenate((X, X_batch))
                            Y = np.concatenate((Y, Y_batch))

                        del X_batch, Y_batch
                        gc.collect()

                        print("Length: ", len(X))

                        if len(X) > SIZE_2k19:


                            X_kitti_batch, Y_kitti_batch = getKittiBatch()
                            print("len kitti:", len(Y_kitti_batch))


                            X = np.concatenate((X, X_kitti_batch))
                            Y = np.concatenate((Y, Y_kitti_batch))
                            
                            del X_kitti_batch, Y_kitti_batch
                            gc.collect()

                            model.fit(X, Y, 
                                        validation_split=VAL_SPLIT, 
                                        batch_size=BATCH_SIZE, 
                                        epochs=EPOCHS, 
                                        shuffle = SHUFFLE,
                                        verbose=2)
                            model.save("drive/MyDrive/Colab Notebooks/commaai/model_final_wo_first.h5")

                            X = np.array([]).reshape(0, IMG_DIM[0], IMG_DIM[1], 2)
                            Y = np.array([])
                            gc.collect()
                except Exception as e:
                    print("ERROR",e)
            gc.collect()



Length:  1197
Length:  2397
Length:  3596
Length:  4795
Length:  5994
Length:  7193
Length:  8392
Length:  9591
Length:  10790
Length:  11987
Length:  13186
Length:  14385
Length:  15584
Length:  16784
Length:  17983
Length:  19181
Length:  20379
Length:  21579
Length:  22778
Length:  23977
Length:  25176
In dir: drive/MyDrive/Colab Notebooks/commaai/data/KITTI/2011_09_26_drive_0064_sync
In dir: drive/MyDrive/Colab Notebooks/commaai/data/KITTI/2011_09_30_drive_0033_sync
In dir: drive/MyDrive/Colab Notebooks/commaai/data/KITTI/2011_09_30_drive_0028_sync
len kitti: 2555
Epoch 1/2
867/867 - 11s - loss: 31.0314
Epoch 2/2
867/867 - 4s - loss: 8.0424
Length:  1199
Length:  2398
Length:  3597
Length:  4796
Length:  5995
Length:  7194
Length:  8393
Length:  9592
Length:  10791
Length:  11990
Length:  13190
Length:  14389
Length:  15588
Length:  16787
Length:  17986
Length:  19186
Length:  20385
Length:  21584
Length:  22783
Length:  23982
Length:  25181
len kitti: 2555
Epoch 1/2
867/867 - 4s -

In [ ]:
getAllKitti()
print("Last kitti batch:", len(Y_kitti))
X = np.concatenate((X, X_kitti))
Y = np.concatenate((Y, Y_kitti))
del X_kitti, Y_kitti
gc.collect()

model.fit(X, Y, 
            validation_split=VAL_SPLIT, 
            batch_size=BATCH_SIZE, 
            epochs=EPOCHS, 
            shuffle = SHUFFLE,
            verbose=2)

del X, Y
gc.collect()

In [10]:
def getImagesFromKittiKITTI(path, size):
    images = []
    prev = cv2.imread(path+"0000000000.png")
    for i in range(1, size):
        try:
            if i >= 2098:
                fn = str(i).zfill(10)+".png"
                next = cv2.imread(path+fn)
                flow = opticalFlow(prev,next, 59, 1125, 0, 375)
                flow = cv2.resize(flow, IMG_DIM, interpolation = cv2.INTER_AREA)
                images.append(flow/127.5 - 1)
                prev = next
        except:
            break
    images = np.array(images)
    return images

def getLabelsFromTXTKITTI(path):
    labels = []
    f = open(path,'r')
    c = 0
    for l in f:
        if c >= 2098:
            labels.append(float(l))
        c+=1
    labels = np.array(labels)
    return labels

folder_id = "2011_10_03_drive_0034_sync"
X_path = root_kitti + folder_id + "/image_03/data/"
Y_path = root_kitti + folder_id + "/labels.txt"
num_photos = len(os.listdir(X_path))

Y_kitti_batch = getLabelsFromTXTKITTI(Y_path)[-2565:]
X_kitti_batch = getImagesFromKittiKITTI(X_path, num_photos)[-2565:]
X_kitti_batch = X_kitti_batch[:,:,:,[0,2]]

Y_kitti_batch*=2
Y_kitti_batch = Y_kitti_batch[:len(X_kitti_batch)]




In [15]:
X = np.array([]).reshape(0, IMG_DIM[0], IMG_DIM[1], 2)
Y = np.array([])

folders = ["99c94dc769b5d96e_2018-07-11--10-10-11/5","99c94dc769b5d96e_2018-07-11--10-10-11/27","99c94dc769b5d96e_2018-07-11--10-10-11/4","99c94dc769b5d96e_2018-07-11--10-10-11/3","99c94dc769b5d96e_2018-07-11--10-10-11/29","99c94dc769b5d96e_2018-07-11--10-10-11/26","99c94dc769b5d96e_2018-07-10--14-21-51/4","99c94dc769b5d96e_2018-07-10--14-21-51/7","99c94dc769b5d96e_2018-07-10--14-21-51/9","99c94dc769b5d96e_2018-07-10--14-21-51/37","99c94dc769b5d96e_2018-07-10--14-21-51/34","99c94dc769b5d96e_2018-07-10--14-21-51/36","99c94dc769b5d96e_2018-07-12--16-42-23/8","99c94dc769b5d96e_2018-07-10--10-01-44/6","99c94dc769b5d96e_2018-07-10--10-01-44/44","99c94dc769b5d96e_2018-07-10--10-01-44/45","99c94dc769b5d96e_2018-07-10--10-01-44/7", "99c94dc769b5d96e_2018-07-10--10-01-44/8"]

for folder in folders:
    path = root_2k19+"Chunk_6/"+folder
    labels_path = path+"/processed_log/CAN/speed/value"
    times_path = path+"/processed_log/CAN/speed/t"
    video_path = path+"/video.hevc"

    Y_batch = getLabelsFrom2k19(labels_path, times_path)
    X_batch = getImagesFrom2k19(video_path)

    X_batch = X_batch[:,:,:,[0,2]]

    Y_batch = Y_batch[:len(X_batch)]

    if len(Y_batch) == len(X_batch):
        X = np.concatenate((X, X_batch))
        Y = np.concatenate((Y, Y_batch))
        print("len:", len(Y))

        del X_batch, Y_batch
        gc.collect()

                        


                            
                            
            

                            

len: 1198
len: 2396
len: 3595
len: 4790
len: 5985
len: 7183
len: 8365
len: 9564
len: 10763
len: 11960
len: 13157
len: 14341
len: 15540
len: 16739
len: 17926
len: 19125
len: 20324
len: 21522


In [18]:
X = np.concatenate((X, X_kitti_batch))
Y = np.concatenate((Y, Y_kitti_batch))

In [20]:
model = tf.keras.models.load_model("drive/MyDrive/Colab Notebooks/commaai/model_final_wo_first.h5")

In [21]:
model.fit(X, Y, 
            validation_split=VAL_SPLIT, 
            batch_size=BATCH_SIZE, 
            epochs=EPOCHS, 
            shuffle = SHUFFLE,
            verbose=2)

Epoch 1/2
753/753 - 9s - loss: 2.2584
Epoch 2/2
753/753 - 5s - loss: 1.4849


In [23]:
del X,Y, X_kitti_batch, Y_kitti_batch
gc.collect()

1309

In [22]:
model.save("drive/MyDrive/Colab Notebooks/commaai/model_final_wo.h5")

In [24]:
X_path_main = "drive/MyDrive/Colab Notebooks/commaai/data/train.mp4"
Y_path_main = "drive/MyDrive/Colab Notebooks/commaai/data/train.txt"

Y_main = getLabelsFromTXT(Y_path_main)
X_main = getImagesFromMain(X_path_main)
Y_main = Y_main[:len(X_main)]

X_main = X_main[:,:,:,[0,2]]

gc.collect()

0

In [25]:
model.evaluate(X_main, Y_main, batch_size=BATCH_SIZE)

638/638 [==============================] - 3s 4ms/step - loss: 19.4461


19.44613265991211

In [27]:
def clacMSE(pred):
    mse = 0
    for i, v in enumerate(pred):
        mse +=  (Y_main[i] - v) * (Y_main[i] - v)
    mse /= len(Y_main)
    return mse

In [28]:
def rollingMean(mylist, rolling_mean):
    if isinstance(mylist, np.ndarray):
        mylist = mylist.tolist()
    avrg = []

    first = 0
    for i in range(rolling_mean):
        first += (mylist[max(i-rolling_mean//2, 0)] / rolling_mean)
    avrg.append(first)
    
    for i in range(1, len(mylist)):
        last = avrg[i-1] * rolling_mean
        past = mylist[max(i-1-rolling_mean//2, 0)]
        new =  mylist[min(i+rolling_mean//2, len(mylist)-1)]
        avrg.append((last - past + new)/rolling_mean)
    return np.array(avrg)

In [29]:
results = model.predict(X_main, batch_size=BATCH_SIZE)
results = results.flatten()
print("MSE clac: ")
print(clacMSE(results))

MSE clac: 
19.446140358473276


In [55]:
# test wich rolling mean value is best

results_mean = rollingMean(results, 80)
print("MSE mean clac: ")
print(clacMSE(results_mean))

MSE mean clac: 
12.639979570176173


In [56]:
test_path = "drive/MyDrive/Colab Notebooks/commaai/data/test.mp4"
X_test = getImagesFromMain(test_path)
X_test = X_test[:,:,:,[0,2]]

In [57]:
predicted_speed = model.predict(X_test, verbose=1).flatten()
predicted_speed_mean = rollingMean(predicted_speed, 40)

338/338 [==============================] - 1s 4ms/step


In [58]:
with open("drive/MyDrive/Colab Notebooks/commaai/data/test_final_wo.txt", "w") as f:
    for speed in predicted_speed:
        f.write(str(speed)+"\n")

with open("drive/MyDrive/Colab Notebooks/commaai/data/test_final_wo_mean.txt", "w") as f:
    for speed in predicted_speed_mean:
        f.write(str(speed)+"\n")

In [59]:
model.fit(X_main, Y_main, 
                    validation_split=VAL_SPLIT, 
                    batch_size=BATCH_SIZE, 
                    epochs=10, 
                    shuffle = SHUFFLE,
                    verbose=2)

Epoch 1/10
638/638 - 4s - loss: 2.2444
Epoch 2/10
638/638 - 4s - loss: 1.1489
Epoch 3/10
638/638 - 4s - loss: 0.9052
Epoch 4/10
638/638 - 4s - loss: 0.7869
Epoch 5/10
638/638 - 4s - loss: 0.7188
Epoch 6/10
638/638 - 4s - loss: 0.6379
Epoch 7/10
638/638 - 4s - loss: 0.5672
Epoch 8/10
638/638 - 4s - loss: 0.5476
Epoch 9/10
638/638 - 4s - loss: 0.5159
Epoch 10/10
638/638 - 4s - loss: 0.4721


In [60]:
predicted_speed = model.predict(X_test, verbose=1).flatten()
predicted_speed_mean = rollingMean(predicted_speed, 40)

338/338 [==============================] - 1s 4ms/step


In [66]:
with open("drive/MyDrive/Colab Notebooks/commaai/data/test_final_w.txt", "w") as f:
    for speed in predicted_speed:
        f.write(str(speed)+"\n")

with open("drive/MyDrive/Colab Notebooks/commaai/data/test_final_w_mean.txt", "w") as f:
    for speed in predicted_speed_mean:
        f.write(str(speed)+"\n")

In [ ]:
while True: pass